In [1]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{ IntegerType, StructField, StructType, StringType, DoubleType}
val sqlContext = new SQLContext(sc)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1606776832569_0003,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.abs
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{min, max, lit}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{IntegerType, StructField, StructType, StringType, DoubleType}
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@68c3ef84


In [2]:
val schema = new StructType().
add("artist_familiarity",DoubleType,true).
add("artist_hotttnesss",DoubleType,true).
add("artist_id",StringType,true).
add("artist_location",StringType,true).
add("artist_mbtags",StringType,true).
add("artist_mbtags_count",StringType,true).
add("artist_name",StringType,true).
add("artist_terms",StringType,true).
add("artist_terms_freq",StringType,true).
add("artist_terms_weight",StringType,true).
add("danceability",DoubleType,true).
add("duration",DoubleType,true).
add("end_of_fade_in",DoubleType,true).
add("energy",DoubleType,true).
add("key",DoubleType,true).
add("key_confidence",DoubleType,true).
add("loudness",DoubleType,true).
add("mode",DoubleType,true).
add("mode_confidence",DoubleType,true).
add("release",StringType,true).
add("segments_confidence",StringType,true).
add("segments_loudness_max",StringType,true).
add("segments_loudness_max_time",StringType,true).
add("segments_pitches",StringType,true).
add("segments_timbre",StringType,true).
add("similar_artists",StringType,true).
add("song_hotttnesss",DoubleType,true).
add("song_id",StringType,true).
add("start_of_fade_out",DoubleType,true).
add("tempo",DoubleType,true).
add("time_signature",IntegerType,true).
add("time_signature_confidence",DoubleType,true).
add("title",StringType,true).
add("track_id",StringType,true).
add("year",IntegerType,true)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema: org.apache.spark.sql.types.StructType = StructType(StructField(artist_familiarity,DoubleType,true), StructField(artist_hotttnesss,DoubleType,true), StructField(artist_id,StringType,true), StructField(artist_location,StringType,true), StructField(artist_mbtags,StringType,true), StructField(artist_mbtags_count,StringType,true), StructField(artist_name,StringType,true), StructField(artist_terms,StringType,true), StructField(artist_terms_freq,StringType,true), StructField(artist_terms_weight,StringType,true), StructField(danceability,DoubleType,true), StructField(duration,DoubleType,true), StructField(end_of_fade_in,DoubleType,true), StructField(energy,DoubleType,true), StructField(key,DoubleType,true), StructField(key_confidence,DoubleType,true), StructField(loudness,DoubleType,tru...

In [3]:
val msdf = sqlContext.read.format("com.databricks.spark.csv").
option("header", "false").
option("delimiter", "\t").
option("inferSchema", "false").
schema(schema).load("s3://millionsongprocessed/*.tsv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]


In [4]:
val usdf= sqlContext.read.format("com.databricks.spark.csv").
option("header","false").
option("delimiter", "\t").
load("s3://millionsongprocessed/train_triplets.txt")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

usdf: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [5]:
usdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---+
|                 _c0|               _c1|_c2|
+--------------------+------------------+---+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|  1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|  1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|  2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|  1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|  1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|  1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|  2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|  1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|  1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|  1|
+--------------------+------------------+---+
only showing top 10 rows



In [6]:
msdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------------------+-----------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------+------+---+--------------+--------+----+---------------+--------------------+--------------------+---------------------+--------------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----------------+-------+--------------+-------------------------+--------------------+------------------+----+
|  artist_familiarity|  artist_hotttnesss|         artist_id|  artist_location|artist_mbtags|artist_mbtags_count|         artist_name|        artist_terms|   artist_terms_freq| artist_terms_weight|danceability| duration|end_of_fade_in|energy|key|key_confidence|loudness|mode|mode_confidence|             release| segments_confidence|segments_loudness_max|segments_loudness_max_time|    segments_pitches|

In [11]:
// 'artist familiarity', 'artist hotttnesss', 'danceability',
//        'duration', 'end of fade in', 'energy', 'key', 'key confidence',
//        'loudness', 'mode', 'mode confidence', 'song hotttnesss',
//        'start of fade out', 'tempo', 'time signature',
//        'time signature confidence', 'year'

var msdf2 = msdf.withColumn("time_signature", when(col("time_signature_confidence")<0.5, 4).otherwise(col("time_signature")))
val msdf3 = msdf2.select($"artist_familiarity",$"duration",$"key", $"key_confidence", 
                         $"loudness",$"mode", $"tempo"/$"time_signature" as "speed", $"song_id")
var msdf4 = msdf3.na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf2: org.apache.spark.sql.DataFrame = [artist_familiarity: double, artist_hotttnesss: double ... 33 more fields]
msdf3: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]
msdf4: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]


In [15]:
msdf4.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---+--------------+--------+----+--------+------------------+
|  artist_familiarity| duration|key|key_confidence|loudness|mode|   speed|           song_id|
+--------------------+---------+---+--------------+--------+----+--------+------------------+
|  0.6399025154955147| 213.9424|1.0|         0.349|  -4.769| 1.0|  24.986|SOBLFFE12AF72AA5BA|
|  0.6717552053402852|118.07302|7.0|         0.187|   -7.24| 1.0|31.36875|SOQPWCR12A6D4FB2A3|
|  0.5817937658450281|218.93179|1.0|         0.736| -11.197| 0.0| 23.0495|SOMZWCG12A8C13C480|
| 0.47694059129745225|193.01832|2.0|          0.76| -13.179| 1.0|10.31975|SOJDASC12A8C13EB49|
|0.013157973056854065|252.94322|6.0|         0.063|  -7.227| 0.0| 40.4975|SOFRDWL12A58A7CEF7|
|  0.5313342110692906|222.35383|5.0|         0.763|  -5.232| 1.0|  25.203|SOZIULX12A8AE46C39|
|  0.6278278253312966|396.35546|7.0|         0.195| -13.225| 1.0|  34.192|SOCSNVI12A8C13ECC2|
|  0.4873567909281477|177.47546|8.0|         0.643|  -9.689|

In [16]:
//feature scaling
val scaledRange = lit(4)
val scaledMin = lit(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

scaledRange: org.apache.spark.sql.Column = 4
scaledMin: org.apache.spark.sql.Column = 1


In [14]:
@transient val (aMin, aMax) = msdf4.agg(min($"artist_familiarity"), max($"artist_familiarity")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val aNorm = ($"artist_familiarity"-aMin)/(aMax-aMin)
@transient val aScal = scaledRange*aNorm + scaledMin

@transient val (dMin, dMax) = msdf4.agg(min($"duration"), max($"duration")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val dNorm = ($"duration"-dMin)/(dMax-dMin)
@transient val dScal = scaledRange*dNorm + scaledMin

@transient val (kMin, kMax) = msdf4.agg(min($"key"), max($"key")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val kNorm = ($"key"-kMin)/(kMax-kMin)
@transient val kScal = scaledRange*kNorm + scaledMin

@transient val (kcMin, kcMax) = msdf4.agg(min($"key_confidence"), max($"key_confidence")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val kcNorm = ($"key"-kcMin)/(kcMax-kcMin)
@transient val kcScal = scaledRange*kcNorm + scaledMin

@transient val (lMin, lMax) = msdf4.agg(min($"loudness"), max($"loudness")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val lNorm = ($"loudness"-lMin)/(lMax-lMin)
@transient val lScal = scaledRange*lNorm + scaledMin

@transient val (mMin, mMax) = msdf4.agg(min($"mode"), max($"mode")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val mNorm = ($"mode"-mMin)/(mMax-mMin)
@transient val mScal = scaledRange*mNorm + scaledMin

@transient val (sMin, sMax) = msdf4.agg(min($"speed"), max($"speed")).first match { case Row(x: Double, y: Double) => (x,y)}
@transient val sNorm = ($"speed"-sMin)/(sMax-sMin)
@transient val sScal = scaledRange*sNorm + scaledMin

val msdf5 = msdf4.withColumn("aScal",aScal).withColumn("dScal", dScal).
            withColumn("kScal", kScal).withColumn("kcScal", kcScal).
            withColumn("lScal", lScal).withColumn("mScal", mScal).
            withColumn("sScal", sScal)
val allDF = msdf5.select("aScal", "dScal", "kScal", "kcScal","lScal","mScal","sScal", "song_id")

// val rowsRDD = allDF.rdd.map(r => (r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3), r.getString(4)))
// rowsRDD.cache()
val vectors = allDF.rdd.map(r => Vectors.dense(r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3),
                                                r.getDouble(4), r.getDouble(5), r.getDouble(6)))
vectors.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aMin: Double = 0.0
aMax: Double = 0.989938569657492
aNorm: org.apache.spark.sql.Column = ((artist_familiarity - 0.0) / 0.989938569657492)
aScal: org.apache.spark.sql.Column = ((4 * ((artist_familiarity - 0.0) / 0.989938569657492)) + 1)
dMin: Double = 0.36526
dMax: Double = 3034.90567
dNorm: org.apache.spark.sql.Column = ((duration - 0.36526) / 3034.54041)
dScal: org.apache.spark.sql.Column = ((4 * ((duration - 0.36526) / 3034.54041)) + 1)
kMin: Double = 0.0
kMax: Double = 11.0
kNorm: org.apache.spark.sql.Column = ((key - 0.0) / 11.0)
kScal: org.apache.spark.sql.Column = ((4 * ((key - 0.0) / 11.0)) + 1)
kcMin: Double = 0.0
kcMax: Double = 1.0
kcNorm: org.apache.spark.sql.Column = ((key - 0.0) / 1.0)
kcScal: org.apache.spark.sql.Column = ((4 * ((key - 0.0) / 1.0)) + 1)
lMin: Double = -57.955
lMax: Double = 4.318
lNorm: org.apache.spark.sql.Column = ((loudness - -57.955) / 62.272999999999996)
lScal: org.apache.spark.sql.Column = ((4 * ((loudness - -57.955) / 62.272999999999996)) + 1)
mMin

In [17]:
vectors.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res37: Array[org.apache.spark.mllib.linalg.Vector] = Array([3.5856251493137155,1.281528154044256,1.3636363636363638,5.0,4.416312045348707,5.0,1.370654423272339], [3.714330872359909,1.155157281296511,3.5454545454545454,29.0,4.257591572591653,5.0,1.4653392275684056], [3.3508277530648085,1.288104952275129,1.3636363636363638,5.0,4.00342042297625,1.0,1.341927444537572], [2.9271522735495337,1.253946936234736,1.7272727272727273,9.0,3.876110031634898,5.0,1.1530881687570926], [1.053166826549274,1.3329373491519922,3.1818181818181817,25.0,4.258426605430925,1.0,1.600759525593194], [3.1469381125462217,1.2926157374849392,2.8181818181818183,21.0,4.38657202961155,5.0,1.3738735063528678], [3.5368354949480074,1.52197716490452,3.5454545454545454,29.0,3.8731553000497807,5.0,1.507220685204827], [2.969240540...

In [18]:
for(k <- 5 to 10)
{
    val kMeansModel = KMeans.train(vectors, k, 20)
    val WSSE = kMeansModel.computeCost(vectors)
    println("K = " + k)
    println("Within set SSE = " + WSSE)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

K = 5
Within set SSE = 8171060.205317439
K = 6
Within set SSE = 5088417.38308835
K = 7
Within set SSE = 4251740.478342662
K = 8
Within set SSE = 3854579.534054612
K = 9
Within set SSE = 3799472.4506969606
K = 10
Within set SSE = 3121875.3910771883


In [19]:
val kMeansModel = KMeans.train(vectors, 10, 20)
kMeansModel.clusterCenters.foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kMeansModel: org.apache.spark.mllib.clustering.KMeansModel = org.apache.spark.mllib.clustering.KMeansModel@5d29011d
[3.0682262914508054,1.339472077732508,4.636363636363635,41.0,4.055611225324538,2.882933836071461,1.4660895669312195]
[3.0739576528183448,1.3133769499408325,1.8065750757726602,9.872325833499259,4.049097695409977,4.12280701754386,1.4640149359496357]
[3.0843689699605568,1.3276610793627461,3.6441550778473575,30.085705856320992,4.084506970621321,5.0,1.4709772118902253]
[3.0800366022731755,1.3185862149213197,2.6286122254708637,18.914734480179504,4.055818187315434,3.2626695506276655,1.4663556471639834]
[3.0908571806632783,1.3247555007118867,4.23382650908534,36.5720915999388,4.096492603022273,3.126383434487683,1.4727898172193463]
[3.0670212279361517,1.3235530285000148,1.0,1.0,4.054546563926648,4.283765434448801,1.4636529696085365]
[3.1256182725235933,1.3418636531149235,1.3636363636363618,5.0,4.1081598816424405,3.9813928378621557,1.4719668214071477]
[3.114239055110017,1.3486212148

In [20]:
val WSSE = kMeansModel.computeCost(vectors)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

WSSE: Double = 3163414.511371173


In [21]:
val predictions = kMeansModel.predict(vectors)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[249] at map at KMeansModel.scala:85


In [22]:
val clusters = predictions.toDF("Cluster")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clusters: org.apache.spark.sql.DataFrame = [Cluster: int]


In [23]:
clusters.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|Cluster|
+-------+
|      6|
|      2|
|      6|
|      1|
|      8|
|      3|
|      2|
|      2|
|      3|
|      4|
|      7|
|      0|
|      1|
|      4|
|      1|
|      2|
|      5|
|      4|
|      4|
|      7|
+-------+
only showing top 20 rows



In [25]:
val df1 = allDF.withColumn("df1Key", monotonically_increasing_id())
val df2 = clusters.withColumn("df2Key", monotonically_increasing_id())
val songclusters = df2.join(df1, $"df1Key" === $"df2Key").drop("df1Key", "df2Key", "aScal", "dScal", "kScal", "kcScal","lScal","mScal","sScal")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df1: org.apache.spark.sql.DataFrame = [aScal: double, dScal: double ... 7 more fields]
df2: org.apache.spark.sql.DataFrame = [Cluster: int, df2Key: bigint]
songclusters: org.apache.spark.sql.DataFrame = [Cluster: int, song_id: string]


In [26]:
songclusters.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|Cluster|           song_id|
+-------+------------------+
|      6|SOMJBYD12A6D4F8557|
|      9|SOMFRKT12A8C146C67|
|      8|SOEAXKZ12A81C208DA|
|      2|SOYOJKY12AB0185DFD|
|      1|SOVHNAQ12AC4689235|
|      3|SONWUSK12AB01837FD|
|      3|SOPKWQK12A6D4F3824|
|      4|SOEHOJX12AB018394D|
|      1|SONBSMJ12AC4687AED|
|      6|SOBLTSM12A6D4F9EEA|
|      2|SOKXFFC12A8C133922|
|      4|SOVBANJ12AAFF444CF|
|      2|SOXWMCD12AB01810DB|
|      7|SONWQQX12AC3DF6762|
|      5|SOGIKFN12AC3DF7E39|
|      6|SOGAJHS12A8C13C37B|
|      1|SOAYOUE12AB01888A6|
|      9|SOYSPCB12A8C141C86|
|      7|SOHPVWT12AB0184E54|
|      5|SOZRRMP12A58A75BEB|
+-------+------------------+
only showing top 20 rows



In [27]:
val list = List("user_id", "SongId", "PlayCount")
import spark.implicits._
val usdf2 = usdf.toDF(list:_*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list: List[String] = List(user_id, SongId, PlayCount)
import spark.implicits._
usdf2: org.apache.spark.sql.DataFrame = [user_id: string, SongId: string ... 1 more field]


In [28]:
usdf2.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+
|             user_id|            SongId|PlayCount|
+--------------------+------------------+---------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|
+--------------------+------------------+---------+
only showing top 10 rows



In [30]:
val userclusters = usdf2.join(songclusters, usdf2("SongId")===songclusters("song_id"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

userclusters: org.apache.spark.sql.DataFrame = [user_id: string, SongId: string ... 3 more fields]


In [31]:
userclusters.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+-------+------------------+
|             user_id|            SongId|PlayCount|Cluster|           song_id|
+--------------------+------------------+---------+-------+------------------+
|2e66366ff3a38f403...|SOAADGZ12A8C1415AC|        1|      8|SOAADGZ12A8C1415AC|
+--------------------+------------------+---------+-------+------------------+
only showing top 1 row



In [32]:
val msdf6 = msdf4.withColumnRenamed("song_id","songID")
msdf6.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdf6: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 6 more fields]
+--------------------+---------+---+--------------+--------+----+--------+------------------+
|  artist_familiarity| duration|key|key_confidence|loudness|mode|   speed|            songID|
+--------------------+---------+---+--------------+--------+----+--------+------------------+
|  0.6399025154955147| 213.9424|1.0|         0.349|  -4.769| 1.0|  24.986|SOBLFFE12AF72AA5BA|
|  0.6717552053402852|118.07302|7.0|         0.187|   -7.24| 1.0|31.36875|SOQPWCR12A6D4FB2A3|
|  0.5817937658450281|218.93179|1.0|         0.736| -11.197| 0.0| 23.0495|SOMZWCG12A8C13C480|
| 0.47694059129745225|193.01832|2.0|          0.76| -13.179| 1.0|10.31975|SOJDASC12A8C13EB49|
|0.013157973056854065|252.94322|6.0|         0.063|  -7.227| 0.0| 40.4975|SOFRDWL12A58A7CEF7|
|  0.5313342110692906|222.35383|5.0|         0.763|  -5.232| 1.0|  25.203|SOZIULX12A8AE46C39|
|  0.6278278253312966|396.35546|7.0|         0.19

In [33]:
val topfeats = userclusters.join(msdf6, userclusters("SongId")===msdf6("songID")).orderBy(desc("PlayCount")).select("Cluster","artist_familiarity","duration","key","key_confidence","loudness","mode","speed")
topfeats.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

topfeats: org.apache.spark.sql.DataFrame = [Cluster: int, artist_familiarity: double ... 6 more fields]
+-------+-------------------+---------+----+--------------+--------+----+-----------------+
|Cluster| artist_familiarity| duration| key|key_confidence|loudness|mode|            speed|
+-------+-------------------+---------+----+--------------+--------+----+-----------------+
|      5| 0.6394469876986911|142.39302| 0.0|         0.622|  -4.076| 1.0|         46.24525|
|      2| 0.6130899696520477|355.36934| 7.0|         0.475|  -7.608| 1.0|         22.98875|
|      4| 0.5449794475871426|162.58567| 9.0|         0.313|  -8.103| 1.0|         21.75625|
|      0| 0.5845937561874122|435.93098|10.0|         0.586|  -7.336| 0.0|28.22066666666667|
|      1|0.49915741844100164|336.74404| 2.0|          0.41| -13.644| 0.0|          27.7055|
|      1|  0.718068955556776|211.64363| 2.0|         0.578|  -8.426| 1.0|36.75566666666666|
|      6| 0.5769307223706501|217.99138| 1.0|         0.234|  -6.024|

In [34]:
val tf = topfeats.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tf: Array[org.apache.spark.sql.Row] = Array([1,0.718068955556776,211.64363,2.0,0.578,-8.426,1.0,36.75566666666666], [2,0.7523647196316754,241.03138,8.0,0.472,-9.731,1.0,37.9235], [5,0.6041098708079196,172.82567,0.0,0.565,-14.026,0.0,25.06525])


In [35]:
val song1 = tf(0)
val song2 = tf(1)
val song3 = tf(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

song1: org.apache.spark.sql.Row = [1,0.718068955556776,211.64363,2.0,0.578,-8.426,1.0,36.75566666666666]
song2: org.apache.spark.sql.Row = [2,0.7523647196316754,241.03138,8.0,0.472,-9.731,1.0,37.9235]
song3: org.apache.spark.sql.Row = [5,0.6041098708079196,172.82567,0.0,0.565,-14.026,0.0,25.06525]


In [36]:
val msdclusters = msdf6.join(songclusters, msdf6("songID")===songclusters("song_id"))
val cluster1 = msdclusters.filter(msdclusters("Cluster")===song1(0))
val cluster2 = msdclusters.filter(msdclusters("Cluster")===song2(0))
val cluster3 = msdclusters.filter(msdclusters("Cluster")===song3(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

msdclusters: org.apache.spark.sql.DataFrame = [artist_familiarity: double, duration: double ... 8 more fields]
cluster1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [artist_familiarity: double, duration: double ... 8 more fields]
cluster2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [artist_familiarity: double, duration: double ... 8 more fields]
cluster3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [artist_familiarity: double, duration: double ... 8 more fields]


In [37]:
cluster1.show()
cluster2.show()
cluster3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------+---+--------------+--------+----+--------+------------------+-------+------------------+
| artist_familiarity| duration|key|key_confidence|loudness|mode|   speed|            songID|Cluster|           song_id|
+-------------------+---------+---+--------------+--------+----+--------+------------------+-------+------------------+
| 0.5473030519864946| 91.11465|2.0|         0.141| -19.722| 1.0| 8.82825|SOATWKO12AC468742B|      1|SOATWKO12AC468742B|
| 0.8489985463716768|201.87383|2.0|           1.0| -12.155| 1.0|  16.276|SOBBGQK12AB0183F1E|      1|SOBBGQK12AB0183F1E|
| 0.6822910077453604|361.50812|2.0|         0.847|  -5.008| 1.0|33.35875|SONBWQH12A6D4FBAA3|      1|SONBWQH12A6D4FBAA3|
| 0.6357570221000797|127.05914|2.0|         0.826| -12.772| 1.0|37.08925|SOFRUDU12A8C13A40C|      1|SOFRUDU12A8C13A40C|
|0.46625786430017835|211.93098|2.0|         0.606|  -4.819| 1.0|22.50425|SOOMECA12A8C132878|      1|SOOMECA12A8C132878|
| 0.4967868356700172|292.17914|2.0|     

In [38]:
val pred1 = cluster1.select($"song_id", abs($"artist_familiarity"-song1(1)) + abs($"duration"-song1(2)) + abs($"key"-song1(3)) + abs($"key_confidence"-song1(4)) + abs($"loudness"-song1(4)) + abs($"mode"-song1(4)) + abs($"speed"-song1(4)) as "score").orderBy(asc("score")).limit(4)
val pred2 = cluster2.select($"song_id", abs($"artist_familiarity"-song2(1)) + abs($"duration"-song2(2)) + abs($"key"-song2(3)) + abs($"key_confidence"-song2(4)) + abs($"loudness"-song1(4)) + abs($"mode"-song1(4)) + abs($"speed"-song1(4)) as "score").orderBy(asc("score")).limit(4)
val pred3 = cluster3.select($"song_id", abs($"artist_familiarity"-song3(1)) + abs($"duration"-song3(2)) + abs($"key"-song3(3)) + abs($"key_confidence"-song3(4)) + abs($"loudness"-song1(4)) + abs($"mode"-song1(4)) + abs($"speed"-song1(4)) as "score").orderBy(asc("score")).limit(4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pred1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]
pred2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]
pred3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [song_id: string, score: double]


In [39]:
pred1.join(msdf, pred1("song_id")===msdf("song_id")).select("score", "title").show()
pred2.join(msdf, pred2("song_id")===msdf("song_id")).select("score", "title").show()
pred3.join(msdf, pred3("song_id")===msdf("song_id")).select("score", "title").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+
|             score|               title|
+------------------+--------------------+
|14.468112928040636|Snowflake Reel/Do...|
|14.361171608985805|      After Midnight|
|12.117066123880496|       Boredom Kills|
|12.228154747031004| Bulerias De Menorca|
+------------------+--------------------+

+------------------+--------------------+
|             score|               title|
+------------------+--------------------+
|14.688653342119725|Pleasure From The...|
|11.632995219711498|What's Done Is Do...|
| 9.538709655698419|If You Tolerate This|
| 15.40362202785942|        Selfish Plan|
+------------------+--------------------+

+------------------+--------------+
|             score|         title|
+------------------+--------------+
| 21.15945812329652|  Harlem Strut|
|21.154301323079107|Screwin' It Up|
|15.719794155079782|    Armageddon|
| 17.18043670924569|     Cable Car|
+------------------+--------------+

